In [ ]:
import numpy as np

import matplotlib.pyplot as plt

import h5py

In [ ]:
w_lya = 1215.67

In [ ]:
filename = '/home/alberto/almacen/VUDS_and_VVDS/vuds/cesam_vuds_dr1.h5'

with h5py.File(filename, "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]

    # Get the data
    spec_f = np.array(f['spectra'])
    spec_w = np.array(f['wavelen'])
    z_Arr = np.array(f['zspec'])
    L_Arr = np.array(f['lyalum'])
    EW_Arr = np.array(f['lya_ew'])
    g_Arr = np.array(f['gJAVA_flux'])[:, 0]
    
mask = (L_Arr > 0) & (g_Arr > 0) & (z_Arr > 1.9) & (z_Arr < 5)

spec_f = spec_f[mask]
z_Arr = z_Arr[mask]
L_Arr = L_Arr[mask]
EW_Arr = EW_Arr[mask]
g_Arr = g_Arr[mask]

# Normalization
# norm = 1. / g_Arr
# spec_f = spec_f * norm.reshape(-1, 1)
# spec_f[np.isnan(spec_f)] = 0.
# print(spec_f.shape)

In [ ]:
# Offset between each source and desired redshift
spec_w_corrected = np.ones(spec_f.shape) * spec_w / (1 + z_Arr).reshape(-1, 1)

w_resolution = 0.1
w_start = spec_w_corrected.min()
w_stop = spec_w_corrected.max()
stack_w = np.arange(w_start, w_stop, w_resolution)

# Normalize the rest-frame spectrum
norm_band_idx = (stack_w > 1500) & (stack_w < 2000)

to_stack_f = np.zeros((len(spec_f), len(stack_w)))
for src in range(len(spec_f)):
    to_stack_f[src] = np.interp(
        stack_w, spec_w_corrected[src], spec_f[src], right=0., left=0.
    )
    norm = np.sum(to_stack_f[src][norm_band_idx] * w_resolution) / (2000 - 1500)
    # if norm == 0: print(z_Arr[src])
    to_stack_f[src] /= norm

# Sum all the fluxes and divide by how many times the flux is not zero in that w bin
w_weights = np.ones(to_stack_f.shape)
w_weights[to_stack_f == 0.] = 0
mask_w = (w_weights > 0)

stack_f = np.zeros(to_stack_f.shape[1])
stack_minus = np.zeros(to_stack_f.shape[1])
stack_plus =  np.zeros(to_stack_f.shape[1])
for w_bin in range(to_stack_f.shape[1]):
    mask = (mask_w[:, w_bin] > 0)
    stack_f[w_bin], stack_minus[w_bin], stack_plus[w_bin] =\
        np.percentile(to_stack_f[:, w_bin][mask], [50, 16, 84])

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

ax.plot(stack_w, stack_f)

ax.set_ylim((-2.5, 2))

plt.savefig('/home/alberto/Desktop/stack', bbox_inches='tight', facecolor='white')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

for src in np.random.choice(range(len(to_stack_f)), 1):
    print(EW_Arr[src])
    ax.plot(stack_w, to_stack_f[src], alpha=0.4, c='dimgray')

ax.plot(stack_w, stack_f, c='red')
ax.plot(stack_w, stack_plus, c='green')
ax.plot(stack_w, stack_minus, c='green')

ax.axvline(1500)
ax.axvline(2000)

ax.set_ylim((-5, 7))
ax.set_xlim((900, 2500))

plt.savefig('/home/alberto/Desktop/stack', bbox_inches='tight', facecolor='white')
plt.show()